In [67]:
from dotenv import load_dotenv
from pydantic_ai import Agent, NativeOutput
from rich import print as rich_print

load_dotenv()

True

In [68]:
import ollama
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.ollama import OllamaProvider

OLLAMA_MODEL_NAME = "gpt-oss:20b"
OLLAMA_BASE_URL = "http://localhost:11434"

ollama.pull(OLLAMA_MODEL_NAME)

model = OpenAIChatModel(
    model_name=OLLAMA_MODEL_NAME,
    provider=OllamaProvider(base_url=f"{OLLAMA_BASE_URL}/v1", api_key="ollama"),
)

In [69]:
naive_agent = Agent(model)

In [70]:
result = await naive_agent.run('give me the names of the first 10 presidents of the united states')
rich_print(result.output)
rich_print(result.usage())

1. George Washington  
2. John Adams  
3. Thomas Jefferson  
4. James Madison  
5. James Monroe  
6. John Quincy Adams  
7. Andrew Jackson  
8. Martin Van Buren  
9. William Henry Harrison  
10. John Tyler

RunUsage(input_tokens=81, output_tokens=207, requests=1)

In [60]:
result = await naive_agent.run('give me the names of the first 10 presidents of the united states')
rich_print(result.output)
rich_print(result.usage())

**The first ten Presidents of the United States**

1. George Washington  
2. John Adams  
3. Thomas Jefferson  
4. James Madison  
5. James Monroe  
6. John Quincy Adams  
7. Andrew Jackson  
8. Martin Van Buren  
9. William Henry Harrison  
10. John Tyler

RunUsage(input_tokens=81, output_tokens=190, requests=1)

In [ ]:
from typing import List
from pydantic import BaseModel


class PresidentialTerm(BaseModel):
    name: str
    term_start_year: int
    term_end_year: int


class Presidents(BaseModel):
    presidents: List[PresidentialTerm]


# Add output type

This one will fail, due to not telling the model the type we expect
while also requiring the output pass validation

In [72]:
default_typed_output_agent = Agent[None, Presidents](
    model,
    output_type=Presidents,
)

In [73]:
result = await default_typed_output_agent.run('give me the names of the first 10 presidents of the united states')

Traceback (most recent call last):
  File "/Users/bradito/git/pydantic-ai-demo/.venv/lib/python3.13/site-packages/pydantic_ai/_tool_manager.py", line 155, in _call_tool
    args_dict = validator.validate_json(call.args or '{}', allow_partial=pyd_allow_partial)
pydantic_core._pydantic_core.ValidationError: 20 validation errors for Presidents
presidents.0.term_start_year
  Field required [type=missing, input_value={'name': 'George Washington'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
presidents.0.term_end_year
  Field required [type=missing, input_value={'name': 'George Washington'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
presidents.1.term_start_year
  Field required [type=missing, input_value={'name': 'John Adams'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
presidents.1.term_end_year
  Field required [type=missing, input_value={

# NativeOutput

Specify to the model that output should be in JSON, of a certain structure

In [ ]:
native_output_agent = Agent[None, Presidents](
    model,
    output_type=NativeOutput(
        Presidents,
        name='Presidents',
        description='A list of presidential terms',
    ),
)

In [62]:
result = await native_output_agent.run('give me the names of the first 10 presidents of the united states')
rich_print(result.output)
rich_print(result.usage())

Presidents(
    presidents=[
        PresidentialTerm(name='George Washington', term_start_year=1789, term_end_year=1797),
        PresidentialTerm(name='John Adams', term_start_year=1797, term_end_year=1801),
        PresidentialTerm(name='Thomas Jefferson', term_start_year=1801, term_end_year=1809),
        PresidentialTerm(name='James Madison', term_start_year=1809, term_end_year=1817),
        PresidentialTerm(name='James Monroe', term_start_year=1817, term_end_year=1825),
        PresidentialTerm(name='John Quincy Adams', term_start_year=1825, term_end_year=1829),
        PresidentialTerm(name='Andrew Jackson', term_start_year=1829, term_end_year=1837),
        PresidentialTerm(name='Martin Van Buren', term_start_year=1837, term_end_year=1841),
        PresidentialTerm(name='William Henry Harrison', term_start_year=1841, term_end_year=1841),
        PresidentialTerm(name='John Tyler', term_start_year=1841, term_end_year=1845)
    ]
)

RunUsage(input_tokens=160, output_tokens=201, requests=1)

In [63]:
rich_print(result.all_messages())

[
    ModelRequest(parts=[UserPromptPart(content='give me the names of the first 10 presidents of the united states',
timestamp=datetime.datetime(2025, 11, 2, 5, 32, 26, 864630, tzinfo=datetime.timezone.utc))]),
    ModelResponse(parts=[ThinkingPart(content="User wants names of first 10 US presidents. Let's list: 1 George 
Washington, 2 John Adams, 3 Thomas Jefferson, 4 James Madison, 5 James Monroe, 6 John Quincy Adams, 7 Andrew 
Jackson, 8 Martin Van Buren, 9 William Henry Harrison, 10 John Tyler. Provide them.", id='reasoning', 
provider_name='ollama'), TextPart(content='{"presidents":[{"name":"George 
Washington","term_start_year":1789,"term_end_year":1797},{"name":"John 
Adams","term_start_year":1797,"term_end_year":1801},{"name":"Thomas 
Jefferson","term_start_year":1801,"term_end_year":1809},{"name":"James 
Madison","term_start_year":1809,"term_end_year":1817},{"name":"James 
Monroe","term_start_year":1817,"term_end_year":1825},{"name":"John Quincy 
Adams","term_start_year":1825,"term_end_year":1829},{"name":"Andrew 
Jackson","term_start_year":1829,"term_end_year":1837},{"name":"Martin Van 
Buren","term_start_year":1837,"term_end_year":1841},{"name":"William Henry 
Harrison","term_start_year":1841,"term_end_year":1841},{"name":"John 
Tyler","term_start_year":1841,"term_end_year":1845}]}\n')], usage=RequestUsage(input_tokens=160, 
output_tokens=201), model_name='gpt-oss:20b', timestamp=datetime.datetime(2025, 11, 2, 5, 32, 35, 
tzinfo=TzInfo(0)), provider_name='ollama', provider_details={'finish_reason': 'stop'}, 
provider_response_id='chatcmpl-745', finish_reason='stop')
]

In [64]:
result = await native_output_agent.run('give me the names of the first 10 presidents of the united states')
rich_print(result.output)

Presidents(
    presidents=[
        PresidentialTerm(name='George Washington', term_start_year=1789, term_end_year=1797),
        PresidentialTerm(name='John Adams', term_start_year=1797, term_end_year=1801),
        PresidentialTerm(name='Thomas Jefferson', term_start_year=1801, term_end_year=1809),
        PresidentialTerm(name='James Madison', term_start_year=1809, term_end_year=1817),
        PresidentialTerm(name='James Monroe', term_start_year=1817, term_end_year=1825),
        PresidentialTerm(name='John Quincy Adams', term_start_year=1825, term_end_year=1829),
        PresidentialTerm(name='Andrew Jackson', term_start_year=1829, term_end_year=1837),
        PresidentialTerm(name='Martin Van Buren', term_start_year=1837, term_end_year=1841),
        PresidentialTerm(name='William Henry Harrison', term_start_year=1841, term_end_year=1841),
        PresidentialTerm(name='John Tyler', term_start_year=1841, term_end_year=1845)
    ]
)

# PromptedOutput

prompt in text that the output should be in a certain JSON structure

In [65]:
from pydantic_ai import PromptedOutput

prompted_output_agent = Agent[None, Presidents](
    model,
    output_type=PromptedOutput(
        Presidents,
        name='Presidents',
        description='A list of presidential terms',
    ),
)

In [66]:
result = await prompted_output_agent.run('give me the names of the first 10 presidents of the united states')
rich_print(result.output)
rich_print(result.usage())
rich_print(result.all_messages())

Presidents(
    presidents=[
        PresidentialTerm(name='George Washington', term_start_year=1789, term_end_year=1797),
        PresidentialTerm(name='John Adams', term_start_year=1797, term_end_year=1801),
        PresidentialTerm(name='Thomas Jefferson', term_start_year=1801, term_end_year=1809),
        PresidentialTerm(name='James Madison', term_start_year=1809, term_end_year=1817),
        PresidentialTerm(name='James Monroe', term_start_year=1817, term_end_year=1825),
        PresidentialTerm(name='John Quincy Adams', term_start_year=1825, term_end_year=1829),
        PresidentialTerm(name='Andrew Jackson', term_start_year=1829, term_end_year=1837),
        PresidentialTerm(name='Martin Van Buren', term_start_year=1837, term_end_year=1841),
        PresidentialTerm(name='William Henry Harrison', term_start_year=1841, term_end_year=1841),
        PresidentialTerm(name='John Tyler', term_start_year=1841, term_end_year=1845)
    ]
)

RunUsage(input_tokens=500, output_tokens=201, requests=1)

[
    ModelRequest(parts=[UserPromptPart(content='give me the names of the first 10 presidents of the united states',
timestamp=datetime.datetime(2025, 11, 2, 5, 32, 46, 259485, tzinfo=datetime.timezone.utc))], instructions='Always 
respond with a JSON object that\'s compatible with this schema:\n\n{"$defs": {"PresidentialTerm": {"properties": 
{"name": {"type": "string"}, "term_start_year": {"type": "integer"}, "term_end_year": {"type": "integer"}}, 
"required": ["name", "term_start_year", "term_end_year"], "title": "PresidentialTerm", "type": "object"}}, 
"properties": {"presidents": {"items": {"$ref": "#/$defs/PresidentialTerm"}, "type": "array"}}, "required": 
["presidents"], "title": "Presidents", "type": "object", "description": "A list of presidential terms"}\n\nDon\'t 
include any text or Markdown fencing before or after.'),
    ModelResponse(parts=[ThinkingPart(content='User asks: "give me the names of the first 10 presidents of the 
united states". According to developer instructions, we must respond with JSON object conforming to schema with 
presidents array each containing name, term_start_year, term_end_year. They didn\'t ask for term years. But we need
to provide them. Probably provide accurate data. First 10 presidents: 1) George Washington 1789-1797; 2) John Adams
1797-1801; 3) Thomas Jefferson 1801-1809; 4) James Madison 1809-1817; 5) James Monroe 1817-1825; 6) John Quincy 
Adams 1825-1829; 7) Andrew Jackson 1829-1837; 8) Martin Van Buren 1837-1841; 9) William Henry Harrison 1841-1841; 
10) John Tyler 1841-1845. That is correct.\n\nWe need to use JSON, no markdown. Return as per schema.\n\nLet\'s 
create JSON: {"presidents":[ {...}, ...]}\n\nMake sure property order is present? Not critical. Include all 
required fields.\n\nReturn.\n\n', id='reasoning', provider_name='ollama'), 
TextPart(content='{"presidents":[{"name":"George 
Washington","term_start_year":1789,"term_end_year":1797},{"name":"John 
Adams","term_start_year":1797,"term_end_year":1801},{"name":"Thomas 
Jefferson","term_start_year":1801,"term_end_year":1809},{"name":"James 
Madison","term_start_year":1809,"term_end_year":1817},{"name":"James 
Monroe","term_start_year":1817,"term_end_year":1825},{"name":"John Quincy 
Adams","term_start_year":1825,"term_end_year":1829},{"name":"Andrew 
Jackson","term_start_year":1829,"term_end_year":1837},{"name":"Martin Van 
Buren","term_start_year":1837,"term_end_year":1841},{"name":"William Henry 
Harrison","term_start_year":1841,"term_end_year":1841},{"name":"John 
Tyler","term_start_year":1841,"term_end_year":1845}]}')], usage=RequestUsage(input_tokens=500, output_tokens=201), 
model_name='gpt-oss:20b', timestamp=datetime.datetime(2025, 11, 2, 5, 33, tzinfo=TzInfo(0)), 
provider_name='ollama', provider_details={'finish_reason': 'stop'}, provider_response_id='chatcmpl-153', 
finish_reason='stop')
]